In [0]:
import numpy as np
import keras
import cv2
from keras import backend as k
from keras.models import Sequential,Model,load_model
from keras.layers import Conv2D,Lambda ,MaxPooling2D, Activation, BatchNormalization, UpSampling2D, Dropout, Flatten, Dense, Input, LeakyReLU, Conv2DTranspose,AveragePooling2D, Concatenate
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import Model
from tensorflow import set_random_seed
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image
from skimage import color, io
from scipy.misc import imresize 
from zipfile import ZipFile 
from io import BytesIO
import matplotlib.pyplot as plt

from keras.applications import vgg16
import tensorflow as tf

np.random.seed(1)
set_random_seed(1)

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  
  def __init__(self, list_IDs, zip_file, batch_size=32, dim=(224, 224), n_channels=2, shuffle=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.list_IDs = list_IDs
    self.zip_file = zip_file
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.on_epoch_end()            # shuffle data on the beginning

  def __len__(self):
    'Denotes the number of batches per epoch'
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    y = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float32)
    X = np.empty((self.batch_size, *self.dim, 3), dtype=np.float32)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
      img_rgb = np.array(Image.open(BytesIO(self.zip_file.read(ID))).convert('RGB'))
      img_rgb = imresize(img_rgb,self.dim)
      img_uv = (color.rgb2yuv(img_rgb)[:,:,1:]*2)+1
      y[i,] = img_uv
      temp = color.rgb2gray(img_rgb)[...,np.newaxis]
      X[i,] = np.concatenate((temp,temp,temp),axis=2)
            
    return X, y

In [0]:
data_path = '/content/drive/My Drive/my_data_b7b_elcima/colored.zip'
zip_data =  ZipFile(data_path, 'r')
ids = zip_data.namelist()[1:]
np.random.shuffle(ids)
dev_length = int(0.1 * len(ids))
test_length = int(0.05 * len(ids))
test_ids = ids[:test_length]
dev_ids = ids[test_length:dev_length]
train_ids = ids[dev_length:]

In [0]:
vgg16_model = vgg16.VGG16(weights='imagenet',include_top = False,input_shape=(224,224,3))
vgg16_model.summary()
layer_dict = dict([(layer.name, layer) for layer in vgg16_model.layers])
x1 = layer_dict['block4_conv3'].output
x1 = Lambda(lambda x: tf.image.resize_bilinear(x,(224,224), align_corners=True))(x1)
x2 = layer_dict['block3_conv3'].output
x2 = Lambda(lambda x: tf.image.resize_bilinear(x,(224,224), align_corners=True))(x2)
x3 = layer_dict['block2_conv2'].output
x3 = Lambda(lambda x: tf.image.resize_bilinear(x,(224,224), align_corners=True))(x3)
x4 = layer_dict['block1_conv1'].output
x5 = vgg16_model.input

x = concatenate([x1, x2, x3 , x4 , x5])

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____

In [0]:
x = Conv2D(128,(3,3),padding='same',strides=(1,1),activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64,(3,3),padding='same',strides=(1,1),activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(2,(3,3),padding='same',strides=(1,1),activation='sigmoid')(x)
x = BatchNormalization()(x)
model = Model(input=vgg16_model.input,output=x) 
for layer in model.layers[:14]:
  layer.trainable = False

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`
  import sys


In [0]:
# compile our model


In [0]:
epochs = 20
batch_size = 16

training_generator = DataGenerator(train_ids, zip_data, batch_size=batch_size)
validation_generator = DataGenerator(dev_ids, zip_data, batch_size=batch_size)

In [0]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/movieColor_{epoch:02d}-{val_loss:.2f}.hdf5')
callbacks_list = [checkpoint]
# training our model
model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs = epochs, callbacks=callbacks_list, verbose=1)
model.save("/content/drive/My Drive/movieColor.h5")

Instructions for updating:
Use tf.cast instead.
Epoch 1/20


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


1970/3749 [==============>...............] - ETA: 34:31 - loss: 0.0627 - acc: 0.5718